In [65]:
%pip install pandas
%pip install plotly==5.21.0
%pip install --upgrade nbformat

import pandas as pd
import plotly.graph_objects as go
import plotly.express as px

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [66]:
pd.set_option("float_format",'{:.2f}'.format)
df = pd.read_csv(filepath_or_buffer='./dados/despesas_22_port_transp.csv', sep=';', low_memory=False)
df = df.drop(columns=["sald_ante","valo_orca","tota_cred","nume_proc_lici"])
df = df[df.columns[:-1]]
df.head(3)

,ano_refe,mes_refe,codi_enti,nome_enti,desc_forc,nume_empe,ano_empe,data_empe,desc_tpem,nome_forn,...,desc_unor,codi_func,codi_subf,codi_prog,codi_acao,desc_acao,codi_sube,desc_sube,ug,cate_desp
0,2022,2,13,Fundo Municipal de Saude de Joao Pessoa,#Transferencias SUS do Governo Federal - Manut...,422,2022,2022-02-09 00:00:00,Global,CENTRAL DE CARGAS ASA BRANCA,...,FUNDO MUNICIPAL DE SAUDE,10,302,5005,386,MAC - REDE HOSPITALAR - MANTER E IMPLEMENTAR O...,33903921,LOCACAO DE IMOVEIS,610095,Outras - Tipo Meta
1,2022,7,13,Fundo Municipal de Saude de Joao Pessoa,#Transferencias SUS do Governo Federal - Manut...,2632,2022,2022-07-18 00:00:00,Global,SISPACK MEDICAL LTDA,...,FUNDO MUNICIPAL DE SAUDE,10,302,5005,386,MAC - REDE HOSPITALAR - MANTER E IMPLEMENTAR O...,33903016,MATERIAL BIOLOGICO,610095,Outras - Tipo Meta
2,2022,7,13,Fundo Municipal de Saude de Joao Pessoa,#Transferencias SUS do Governo Federal - Manut...,2631,2022,2022-07-18 00:00:00,Global,SISPACK MEDICAL LTDA,...,FUNDO MUNICIPAL DE SAUDE,10,302,5005,387,MAC - ACOES DE MEDIA E ALTA COMPLEXIDADE - MAN...,33903016,MATERIAL BIOLOGICO,610095,Outras - Tipo Meta


In [67]:
df["data_empe"]=pd.to_datetime(df["data_empe"])
df["data_movi"]=pd.to_datetime(df["data_movi"])
df["data_ulal"]=pd.to_datetime(df["data_ulal"])
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 186215 entries, 0 to 186214
Data columns (total 41 columns):
 #   Column     Non-Null Count   Dtype         
---  ------     --------------   -----         
 0   ano_refe   186215 non-null  int64         
 1   mes_refe   186215 non-null  int64         
 2   codi_enti  186215 non-null  int64         
 3   nome_enti  186215 non-null  object        
 4   desc_forc  186215 non-null  object        
 5   nume_empe  186215 non-null  int64         
 6   ano_empe   186215 non-null  int64         
 7   data_empe  186215 non-null  datetime64[ns]
 8   desc_tpem  186215 non-null  object        
 9   nome_forn  186215 non-null  object        
 10  codi_elem  186215 non-null  int64         
 11  desc_desp  186215 non-null  object        
 12  desc_orga  186215 non-null  object        
 13  desc_tpde  186215 non-null  object        
 14  nume_proc  43720 non-null   object        
 15  desc_tpli  186215 non-null  object        
 16  nume_lici  42840 non

In [68]:
valores_unicos = pd.DataFrame(map(lambda columnn:
    {
        "coluna": columnn,
        "quantidade": df[columnn].nunique(),
        "valores": df[columnn].unique()
    } ,df.columns))
valores_unicos.sort_values("quantidade")

,coluna,quantidade,valores
0,ano_refe,1,[2022]
6,ano_empe,1,[2022]
8,desc_tpem,3,"[Global, Ordinario, Estimativo]"
19,desc_tpmo,6,"[Emissao de Empenho, Anulacao de Empenho, Liqu..."
40,cate_desp,10,"[Outras - Tipo Meta, Obras - Tipo Meta TCE, De..."
1,mes_refe,12,"[2, 7, 3, 4, 5, 6, 8, 9, 10, 11, 1, 12]"
3,nome_enti,17,"[Fundo Municipal de Saude de Joao Pessoa, Pref..."
2,codi_enti,17,"[13, 1, 2, 10, 42, 23, 24, 14, 34, 18, 6, 33, ..."
15,desc_tpli,17,"[Dispensa de Licitacao por outros motivos, Pre..."
39,ug,17,"[610095, 201095, 304095, 603095, 301095, 30309..."


In [69]:
movimentacao_por_tipo = df[["desc_tpmo","valo_movi"]].groupby("desc_tpmo").sum()
movimentacao_por_tipo.sort_values("valo_movi")

,valo_movi
desc_tpmo,
Emissao de Empenho,-3632785698.71
Estorno de Liquidacao de Empenho,-180677272.77
Estorno de Pagamento de Empenho,-8333655.53
Anulacao de Empenho,481635157.16
Pagamento de Empenho,2354546886.69
Liquidacao de Empenho,3011004671.81


In [70]:
pagamentos_empenho = df[df["desc_tpmo"]=="Pagamento de Empenho"]
pagamentos_empenho.head(3)

,ano_refe,mes_refe,codi_enti,nome_enti,desc_forc,nume_empe,ano_empe,data_empe,desc_tpem,nome_forn,...,desc_unor,codi_func,codi_subf,codi_prog,codi_acao,desc_acao,codi_sube,desc_sube,ug,cate_desp
98250,2022,5,6,Instituto de Previdencia do Municipio de Joao ...,#Recursos vinculados ao RPPS - Fundo em Capita...,336,2022,2022-05-20,Ordinario,VINICIUS ALMEIDA VIEIRA,...,INSTITUTO DE PREVIDENCIA DO MUNICIPIO - IPM,9,122,5001,66,MANTER E IMPLEMENTAR OS SERVICOS ADMINISTRATIV...,33909309,RESTITUICOES A SERVIDORES,302095,Outras - Tipo Meta
98251,2022,5,13,Fundo Municipal de Saude de Joao Pessoa,#Transferencias SUS do Governo Federal - Manut...,1738,2022,2022-05-27,Ordinario,WHITE MARTINS GASES INDUSTRIAIS DO NORDESTE LTDA,...,FUNDO MUNICIPAL DE SAUDE,28,846,7001,391,ENCARGOS COM INDENIZACOES E RESTITUICOES - FMS,33909312,INDENIZACOES E RESTITUICOES,610095,Outras - Tipo Meta
98252,2022,5,13,Fundo Municipal de Saude de Joao Pessoa,#Transferencias SUS do Governo Federal - Manut...,1740,2022,2022-05-27,Ordinario,WHITE MARTINS GASES INDUSTRIAIS DO NORDESTE LTDA,...,FUNDO MUNICIPAL DE SAUDE,28,846,7001,391,ENCARGOS COM INDENIZACOES E RESTITUICOES - FMS,33909312,INDENIZACOES E RESTITUICOES,610095,Outras - Tipo Meta


In [71]:
pagamentos_empenho = df[(df["desc_tpmo"]=="Pagamento de Empenho") & (df["valo_movi"]>0)]

In [72]:
# Gráfico tipo KPI
total_pagamento_empenho = pagamentos_empenho["valo_movi"].sum()
total_pagamento_empenho_kpi = go.Figure(go.Indicator(
    mode = "number",
    value = total_pagamento_empenho,
    number = {'prefix': "R$ "},
    title="Total Pagamento Empenho",
    ))

total_pagamento_empenho_kpi.update_layout(paper_bgcolor="lightgray",
                  width=350,
                  height=250)
total_pagamento_empenho_kpi.show()

In [73]:
movimentacao_por_tipo = pagamentos_empenho[["desc_func","valo_movi"]].groupby("desc_func").sum().reset_index()
movimentacao_por_tipo= movimentacao_por_tipo.sort_values("valo_movi", ascending=False)
movimentacao_por_tipo

,desc_func,valo_movi
16,Saude,825740098.99
8,Educacao,485210221.80
0,Administracao,324467014.16
14,Previdencia Social,288889280.41
9,Encargos Especiais,169766722.81
20,Urbanismo,146210258.46
10,Energia,36947107.74
1,Assistencia Social,36714090.32
17,Seguranca Publica,22680757.40
19,Transporte,20359479.40


In [89]:
totais_por_funcao_bar = px.bar(movimentacao_por_tipo,
             y='valo_movi',
             x='desc_func',
             text_auto='.1s',
             title="Totais por função",
             labels={
                 'valo_movi':'movimentação',
                 'desc_func':'função'
                 })
totais_por_funcao_bar.update_traces(textfont_size=12, 
                  textangle=0, 
                  textposition="outside", 
                  cliponaxis=False)
totais_por_funcao_bar.show()

In [111]:
# Gráfico tipo KPI
total_pagamento_saude = movimentacao_por_tipo[movimentacao_por_tipo["desc_func"]=="Saude"]["valo_movi"].values[0]
total_pagamento_educacao = movimentacao_por_tipo[movimentacao_por_tipo["desc_func"]=="Educacao"]["valo_movi"].values[0]

total_pagamento_saude_kpi = go.Figure(go.Indicator(
    mode = "number",
    value = total_pagamento_saude,
    number = {'prefix': "R$ "},
    title="Total Pagamento Empenho",
    ))

total_pagamento_saude_kpi.update_layout(paper_bgcolor="lightgray",
                  width=350,
                  height=250)
total_pagamento_saude_kpi.show()

In [96]:
subfuncoes_saude = pagamentos_empenho[pagamentos_empenho["desc_func"]=="Saude"][["desc_subf","valo_movi"]].groupby(["desc_subf"]).sum().reset_index()
subfuncoes_saude = subfuncoes_saude.sort_values("valo_movi", ascending=False)
subfuncoes_saude_bar = px.bar(subfuncoes_saude,
             y='valo_movi',
             x='desc_subf',
             text_auto='.1s',
             title="Totais por Sub Funções da Saúde",
             labels={
                 'valo_movi':'movimentação',
                 'desc_subf':'subfunção'
                 })
subfuncoes_saude_bar.update_traces(textfont_size=12, 
                  textangle=0, 
                  textposition="outside", 
                  cliponaxis=False)
subfuncoes_saude_bar.show()

In [97]:
subfuncoes_educacao = pagamentos_empenho[pagamentos_empenho["desc_func"]=="Educacao"][["desc_subf","valo_movi"]].groupby(["desc_subf"]).sum().reset_index()
subfuncoes_educacao = subfuncoes_educacao.sort_values("valo_movi", ascending=False)
subfuncoes_educacao_bar = px.bar(subfuncoes_educacao,
             y='valo_movi',
             x='desc_subf',
             text_auto='.1s',
             title="Totais por Sub Funções da Educação",
             labels={
                 'valo_movi':'movimentação',
                 'desc_subf':'subfunção'
                 })
subfuncoes_educacao_bar.update_traces(textfont_size=12, 
                  textangle=0, 
                  textposition="outside", 
                  cliponaxis=False)
subfuncoes_educacao_bar.show()